In [4]:
import numpy as np
import pandas as pd
import os
from toolsets.file_io import readin_peak_list
from feature_finding.feature_finding_alphapept import find_feature, find_feature_alphapept
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
import toolsets.spectra_operations as so
from toolsets.search import quick_search_values

In [5]:
alphapept_folder = '/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/alphapept_features/20230206_PHV_H2L_1870AX_M5P2_M7P1/pos'
msdial_folder = '/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/peak_list/20230206_PHV_H2L_1870AX_M5P2_M7P1/pos'
mzml_folder = '/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/MZMLS/20230206_PHV_H2L_1870AX_M5P2_M7P1/pos'

In [49]:
file_list = []
for root, dirs, files in os.walk(msdial_folder):
    for file in files:
        # print(file)

        if file.endswith('.txt'):
            file_list.append(file.split('.')[0])

In [50]:
len(file_list)

39

In [7]:
istd_info = pd.read_csv('/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/istd_info.csv')


In [93]:
# istd_info.to_csv('/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/istd_info.csv', index=False)

In [8]:
intensity_dir = '/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/intensity_analysis'

In [51]:
Fill_percent = []
pmz = []
rt = []
intensity_cv = []

for index, row in tqdm(istd_info.iterrows(), total = len(istd_info)):

    FOUND = []
    fill_percent = []
    mz = []
    mz_offset =[]
    rt = []

    # m_rt_offset = []
    isotope = []
    intensity = []
    for file in file_list:
        msdial_result_temp = readin_peak_list(os.path.join(msdial_folder, file+'.txt'))
        msdial_feature = find_feature(msdial_result_temp, row['[M+H]+'], row['RT_suggested'], mz_column='Precursor m/z', rt_column='RT (min)', intensity_column='Height')
        if len(msdial_feature)>0:
            FOUND.append(True)
            mz.append(msdial_feature.iloc[0]['Precursor m/z'])
            mz_offset.append(np.abs(msdial_feature.iloc[0]['Precursor m/z']-row['[M+H]+']))
            rt.append(msdial_feature.iloc[0]['RT (min)'])
            # m_rt_offset.append(np.abs(msdial_feature.iloc[0]['RT (min)']-row['RT_suggested']))
            isotope.append(msdial_feature.iloc[0]['Isotope'])
            intensity.append(msdial_feature.iloc[0]['Height'])
        else:
            FOUND.append(False)
            mz.append(np.NAN)
            mz_offset.append(np.NAN)
            rt.append(np.NAN)
            # rt_offset.append(np.NAN)
            isotope.append(np.NAN)
            intensity.append(np.NAN)
    intensity_cv.append(np.round(pd.Series(intensity).std()/pd.Series(intensity).mean()*100, 1))
    Fill_percent.append(sum(bool(s) for s in FOUND)/len(FOUND))
    # break

summary_statistic = pd.DataFrame(list(zip(istd_info['compound_name'], Fill_percent,intensity_cv)),
                                 columns =['istd', 'fill_percent', 'intensity_cv']
                                      )

100%|██████████| 17/17 [00:37<00:00,  2.19s/it]


In [53]:
istd_info

,compound_name,mono_mass,RT_suggested,[M+H]+
0,C4-APS,215.061614,1.18,216.068890
1,C5-APS,229.077265,2.13,230.084541
2,C6-APS,243.092915,2.84,244.100191
3,C7-APS,257.108565,3.42,258.115841
4,C8-APS,271.124215,4.00,272.131491
5,C9-APS,285.139865,4.46,286.147141
6,C10-APS,299.155515,4.91,300.162791
7,C11-APS,313.171165,5.28,314.178441
8,C12-APS,327.186815,5.68,328.194091
9,C13-APS,341.202465,6.03,342.209741
